# Скрапинг Петровича

In [149]:
import xlsxwriter
from selenium.webdriver import Firefox
#from selenium.webdriver.common.by import By
#from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webelement import WebElement
#import selenium.common.exceptions as selEx
#from selenium.webdriver import ActionChains
#from collections import defaultdict
import time
import string
import json
from io import BytesIO
from urllib.request import urlopen
import urllib.error as urEr

In [100]:
wbName = r'C:\Users\25430\Notebook_test\python-scraping\ремонт 70м2.xlsx'

In [101]:
browser = Firefox()
browser.get("https://moscow.petrovich.ru/estimate/11673276/")
browser.set_window_size(1000, 1100) # maximize_window()
# browser.implicitly_wait(1)

In [102]:
def toint(s: str) -> int:
    s = s.strip(string.whitespace).replace(' ', '').replace(',', '.')
    if s:
        return int(s)
    else:
        return 0

def tofloat(s: str) -> float:
    s = s.strip(string.whitespace).replace(' ', '').replace(',', '.')
    if s:
        return float(s)
    else:
        return 0.0

In [103]:
def getProduct(product: WebElement) -> dict:
    resdict = {}
    browser.execute_script("arguments[0].scrollIntoView();", product)
    art = product.find_element_by_css_selector('span[data-test="product-code"]').text
    art = toint(art)
    resdict[art] = {}

    img = product.find_element_by_class_name('listed-product-img').get_attribute('src')
    resdict[art]['img'] = img
    prname = product.find_element_by_css_selector('span[data-test="product-title"]').text
    resdict[art]['name'] = prname.strip(string.whitespace)
    price = product.find_element_by_css_selector('span[data-test="product-retail-price"]').text
    resdict[art]['price'] = tofloat(price)
    quant = product.find_element_by_css_selector('input[data-test="product-counter"]')
    resdict[art]['quant'] = toint(quant.get_attribute('value'))
    link = product.find_element_by_class_name('listed-product-link').get_attribute('href')
    resdict[art]['link'] = link
    
    return resdict

In [104]:
def getName(elem: WebElement) -> str:
    names = elem.find_elements_by_tag_name('input')
    if names:
        name = names[0].get_attribute('name').strip()
    else:
        name = ""
    return name

In [105]:
button = browser.find_element_by_css_selector('button[data-test="renovation-tab"]')
button.click()

In [106]:
home = {}
# все комнаты
rooms = browser.find_elements_by_css_selector('div[data-test="room-dropdown"]')
for i, room in enumerate(rooms):
    room.click()
    name ="{}. {}".format(i, getName(room))
    print(name)
    home[name] = {}
    # черновые/отделочные материалы
    drafts = room.find_elements_by_css_selector('div[data-test="draft-dropdown"]')
    if drafts:
        for draft in drafts:
            # browser.execute_script("arguments[0].scrollIntoView();", draft)
            draft.click()
            draftname = getName(draft)
            print("\t" + draftname)
            home[name][draftname] = {}
            # части комнаты
            rparts = draft.find_elements_by_css_selector('div[data-test="room-part-dropdown"]')
            for rpart in rparts:    
                rpart.click()
                rpartname = getName(rpart)
                print("\t\t" + rpartname)
                home[name][draftname][rpartname] = {}
                # продукты    
                products = rpart.find_elements_by_css_selector('div[data-test="product-block"]')
                for product in products:
                    for k, v in getProduct(product).items():
                        home[name][draftname][rpartname][k] = v
    else:
        home[name][name] = {}
        home[name][name][name] = {}
        products = room.find_elements_by_css_selector('div[data-test="product-block"]')
        for product in products:
            for k, v in getProduct(product).items():
                home[name][name][name][k] = v
        
# home

0. Детская
	Черновые
		Пол
		Стены
		Потолок
	Отделочные
		Пол
		Напольный плинтус
		Стены
		Потолок
		Дверь
		Электрика
1. Кухня
	Черновые
		Пол
		Стены
		Потолок
	Отделочные
		Пол
		Напольный плинтус
		Стены
		Потолок
		Дверь
		Электрика
2. Санузел
	Черновые
		Пол
		Стены
		Потолок
	Отделочные
		Стены+пол
		Потолок
		Дверь
		Мебель для ванны
		Ванна
		Унитаз
		Электрика
3. Прихожая
	Черновые
		Пол
		Стены
		Потолок
	Отделочные
		Пол
		Напольный плинтус
		Стены
		Потолок
		Дверь
		Электрика
4. Спальня
	Черновые
		Пол
		Стены
		Потолок
	Отделочные
		Пол
		Напольный плинтус
		Стены
		Потолок
		Дверь
		Электрика
5. Детская
	Черновые
		Пол
		Стены
		Потолок
	Отделочные
		Пол
		Напольный плинтус
		Стены
		Потолок
		Дверь
		Электрика
6. Расходные материалы
7. Сопутствующие товары


{'0. Детская': {'Черновые': {'Пол': {504677: {'img': 'https://cs.petrovich.ru/images/150192/original-100x100-fit.jpg',
     'name': 'Ровнитель (наливной пол) для звукоизоляционных полов Weber.vetonit 4350 самовыравнивающийся 25 кг',
     'price': 769.0,
     'quant': 22,
     'link': 'https://moscow.petrovich.ru/product/504677/'},
    631471: {'img': 'https://cs.petrovich.ru/images/2497991/original-100x100-fit.jpg',
     'name': 'Грунт Ceresit CT17 концентрат 1 л',
     'price': 598.0,
     'quant': 1,
     'link': 'https://moscow.petrovich.ru/product/631471/'},
    104493: {'img': 'https://cs.petrovich.ru/images/2963685/original-100x100-fit.jpg',
     'name': 'Лента Knauf FE кромочная демпферная для пола 8х100 мм 40 м',
     'price': 1049.0,
     'quant': 1,
     'link': 'https://moscow.petrovich.ru/product/104493/'}},
   'Стены': {613777: {'img': 'https://cs.petrovich.ru/images/1256228/original-100x100-fit.jpg',
     'name': 'Штукатурка гипсовая КМ Теплослой белая 30 кг',
     'price

In [107]:
with open(wbName.replace("xlsx", "json"), "w", encoding = "utf8") as write_file:
    json.dump(home, write_file, ensure_ascii=False)

In [108]:
browser.close()

In [141]:
fbold = {
            'bold': 1,
            'align': 'center',
            'valign': 'vcenter',
            'border': 1,
            'text_wrap': True
        }
fnorm = {
            'valign': 'vcenter',
            'border': 1,
            'text_wrap': True
        }
# Light red fill with dark red text.
fcondred = {
            'bg_color':   '#FFC7CE', 
            'font_color': '#9C0006'
        }

In [156]:
workbook = xlsxwriter.Workbook(wbName)
cfbold = workbook.add_format(fbold)
cfnorm = workbook.add_format(fnorm)
cfcondred = workbook.add_format(fcondred)
wsCur = workbook.add_worksheet("70m2")
wsCur.set_default_row(108)
wsCur.set_column('A:C', 22.7)
wsCur.set_column('D:D', 7.5)
wsCur.set_column('E:E', 19) 
wsCur.set_column('F:F', 60) 
wsCur.set_column('G:H', 7.5)
wsCur.set_column('I:I', 39.3)
wsCur.set_column('J:J', 8)

wsCur.write(0, 0, "Помещение", cfbold)
wsCur.write(0, 1, "Отделка", cfbold)
wsCur.write(0, 2, "Элемент", cfbold)
wsCur.write(0, 3, "Артикул", cfbold)
wsCur.write(0, 4, "Картинка", cfbold)
wsCur.write(0, 5, "Наименование", cfbold)
wsCur.write(0, 6, "Цена", cfbold)
wsCur.write(0, 7, "Кол-во", cfbold)
wsCur.write(0, 8, "Ссылка", cfbold)
wsCur.write(0, 9, "Сумма", cfbold)

0

In [157]:
def writeDict(k, v, c, prev: list = []):
    global r
    isDict = True
    if isinstance(v,dict):
        wsCur.write(r, c, k, cfnorm)        
        for k1, v1 in v.items():
            writeDict(k1, v1, c + 1, prev + [k])
            
            if not isinstance(v1, dict):
                c += 1
                isDict = False
                
        if not isDict:
            for i in range(len(prev)):
                wsCur.write(r, i, prev[i], cfnorm)
            wsCur.write_formula(r, 9, "=G{0} * H{0}".format(r+1), cfnorm)
            r += 1
    else:
        if c == 4:
            try:
                image_data = BytesIO(urlopen(v).read())
                wsCur.insert_image(r, c, v, {'image_data': image_data, 'object_position': 2})
            except (urEr.HTTPError, urEr.URLError) as e:
                print(v)
                print(e)
                wsCur.write_url(r, c, v)
        elif c == 8:
            wsCur.write_url(r, c, v)
        else:
            wsCur.write(r, c, v, cfnorm)   

In [158]:
r = 1
for k, v in home.items():
    writeDict(k, v, 0)
wsCur.conditional_format('G1:G{}'.format(r), {'type': 'cell', 'criteria': '=', 'value': 0, 'format': cfcondred})
wsCur.conditional_format('J1:J{}'.format(r), {'type': 'cell', 'criteria': '=', 'value': 0, 'format': cfcondred})
workbook.close()

https://cs.petrovich.ru/image/4643606/original-100x100-fit.jpg
HTTP Error 404: Not Found
https://cs.petrovich.ru/image/4673012/original-100x100-fit.jpg
HTTP Error 404: Not Found
https://cs.petrovich.ru/image/4666960/original-100x100-fit.jpg
HTTP Error 404: Not Found
https://cs.petrovich.ru/image/4644470/original-100x100-fit.jpg
HTTP Error 404: Not Found
https://cs.petrovich.ru/image/4649950/original-100x100-fit.jpg
HTTP Error 404: Not Found


In [159]:
type(wsCur)

xlsxwriter.worksheet.Worksheet